In [ ]:
%matplotlib qt

In [ ]:
# %matplotlib widget
import os, numpy as np, imageio.v2 as imageio
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import cv2

IMG_PATH = r'./p1_amp.png'
OUT_DIR  = r'./img'; os.makedirs(OUT_DIR, exist_ok=True)

def imread_gray01(p):
    im = imageio.imread(p)
    if im.ndim==3: im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    im = im.astype(np.float32); 
    if im.max()>1: im/=255.0
    return im

def circshift_rows_var(img, shifts):
    H,W = img.shape; out = np.empty_like(img)
    for y in range(H):
        s = int(shifts[y] % W)
        out[y] = img[y] if s==0 else np.r_[img[y, s:], img[y, :s]]
    return out

def circshift_cols_var(img, shifts):
    H,W = img.shape; out = np.empty_like(img)
    for x in range(W):
        s = int(shifts[x] % H)
        col = img[:,x]; out[:,x] = col if s==0 else np.r_[col[s:], col[:s]]
    return out

def reassemble_tilted(img, c0, slopeC, r0, slopeR):
    H,W = img.shape
    ys = np.arange(H, dtype=float); xs = np.arange(W, dtype=float)
    c_y = np.round(c0 + slopeC*(ys-1)).astype(int); c_y = (c_y-1)%W + 1
    r_x = np.round(r0 + slopeR*(xs-1)).astype(int); r_x = (r_x-1)%H + 1
    I1 = circshift_rows_var(img, (c_y-1))  
    I2 = circshift_cols_var(I1, (r_x-1))   
    return I2

im0 = imread_gray01(IMG_PATH); H,W = im0.shape

plt.close('all')
fig, ax = plt.subplots(figsize=(6,6))   
plt.subplots_adjust(bottom=0.32)

c0_init = W/2; r0_init = H/2; slopeC_init = 0.45; slopeR_init = 0.00
im_show = reassemble_tilted(im0, c0_init, slopeC_init, r0_init, slopeR_init)
h_im = ax.imshow(im_show, cmap='gray', vmin=0, vmax=1)
ax.axis('image'); ax.axis('off')
ax.set_title(f'c0={c0_init:.1f}, mC={slopeC_init:.3f} | r0={r0_init:.1f}, mR={slopeR_init:.3f}')

ax_c0 = plt.axes([0.15,0.24,0.7,0.03])
ax_mC = plt.axes([0.15,0.19,0.7,0.03])
ax_r0 = plt.axes([0.15,0.14,0.7,0.03])
ax_mR = plt.axes([0.15,0.09,0.7,0.03])
sl_c0 = Slider(ax_c0, 'c0 (col)',      1, W,  valinit=c0_init)
sl_mC = Slider(ax_mC, 'slopeC col/row',-1, 1,  valinit=slopeC_init)
sl_r0 = Slider(ax_r0, 'r0 (row)',      1, H,  valinit=r0_init)
sl_mR = Slider(ax_mR, 'slopeR row/col',-1, 1,  valinit=slopeR_init)

def update(_):
    c0 = sl_c0.val; mC = sl_mC.val; r0 = sl_r0.val; mR = sl_mR.val
    h_im.set_data(reassemble_tilted(im0, c0, mC, r0, mR))
    ax.set_title(f'c0={c0:.1f}, mC={mC:.3f} | r0={r0:.1f}, mR={mR:.3f}')
    fig.canvas.draw_idle()

for sl in (sl_c0, sl_mC, sl_r0, sl_mR): sl.on_changed(update)

ax_btn = plt.axes([0.83,0.03,0.12,0.05]); btn=Button(ax_btn,'Save')
def on_save(_):
    c0 = sl_c0.val; mC = sl_mC.val; r0 = sl_r0.val; mR = sl_mR.val
    out = reassemble_tilted(im0, c0, mC, r0, mR)
    base=os.path.splitext(os.path.basename(IMG_PATH))[0]
    path=os.path.join(OUT_DIR, f'{base}_tilt_c{c0:.1f}_mC{mC:+.3f}_r{r0:.1f}_mR{mR:+.3f}.png')
    imageio.imwrite(path, (np.clip(out,0,1)*255).astype(np.uint8))
    print('Saved:', path)
btn.on_clicked(on_save)
plt.show()


Saved: ./out\3f_6.5m_tilt_c53.1_mC-0.104_r402.4_mR-0.001.png
